In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [3]:
cd MyDrive/TargetClassify/

/gdrive/MyDrive/TargetClassify


In [4]:
ls

DataSet/
DataSet.zip
model.pth
resnet50_weights_tf_dim_ordering_tf_kernels_notop1.h5
resnet50_weights_tf_dim_ordering_tf_kernels_notop2.h5
resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
resnet50_weights_tf_dim_ordering_tf_kernels_notop.zip
Untitled
Untitled.ipynb


In [5]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, ResNet50
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
from keras import backend as K
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.models import load_model


K.set_image_data_format('channels_first')
img_width, img_height = 200,200


input_tensor = Input(shape=(3,img_width, img_height))
model = ResNet50(include_top=False, weights=None, input_tensor=input_tensor)
print('Model loaded.')
model.load_weights(WEIGHTS_PATH_NO_TOP)


x = model.output
x = Flatten()(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation = 'softmax')(x)

model2 = Model(inputs=model.input, outputs=x)


for layer in model2.layers[:45]: # set the first 11 layers(fine tune conv4 and conv5 block can also further improve accuracy
    layer.trainable = False
model2.compile(loss='binary_crossentropy',
              optimizer = SGD(lr=1e-3,momentum=0.9),
              metrics=['accuracy'])



Model loaded.


In [7]:
train_data_dir = './DataSet/test'
validation_data_dir = './DataSet/train'

nb_train_samples = 2536
nb_validation_samples = 2636
epochs = 200
batch_size = 32


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range=10.,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# 图片generator
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model2.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Found 2636 images belonging to 10 classes.
Found 2536 images belonging to 10 classes.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
79/79 [==============================] - 523s 7s/step - loss: 0.3080 - accuracy: 0.3042 - val_loss: 0.3553 - val_accuracy: 0.1341
Epoch 2/200
79/79 [==============================] - 35s 442ms/step - loss: 0.2418 - accuracy: 0.5048 - val_loss: 0.3569 - val_accuracy: 0.1830
Epoch 3/200
79/79 [==============================] - 34s 433ms/step - loss: 0.1891 - accuracy: 0.6412 - val_loss: 0.3518 - val_accuracy: 0.2275
Epoch 4/200
79/79 [==============================] - 34s 425ms/step - loss: 0.1517 - accuracy: 0.7294 - val_loss: 0.2455 - val_accuracy: 0.5134
Epoch 5/200
79/79 [==============================] - 32s 411ms/step - loss: 0.1138 - accuracy: 0.8138 - val_loss: 0.1577 - val_accuracy: 0.6968
Epoch 6/200
79/79 [==============================] - 32s 410ms/step - loss: 0.0899 - accuracy: 0.8684 - val_loss: 0.0860 - val_accuracy: 0.8580
Epoch 7/200
79/79 [==============================] - 32s 411ms/step - loss: 0.0695 - accuracy: 0.8975 - val_loss: 0.0712 - val_accuracy: 0

KeyboardInterrupt: ignored